# Using YoLo v5 network

In this workbook, we use YoLo (You only Look once) network from Hugging Face to detect objects. 

In [ ]:
!pip install --upgrade numpy

In [ ]:
# install YoLo v5 network
!pip install -q -U yolov5

In [ ]:
import yolov5

# load model
model = yolov5.load('fcakyon/yolov5s-v7.0')
  
# set model parameters
model.conf = 0.25  # NMS confidence threshold
model.iou = 0.45  # NMS IoU threshold
model.agnostic = False  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.max_det = 1000  # maximum number of detections per image

# set image
img = 'https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg'

# Load and preprocess the image
image = Image.open('./test_image.jpg')
transform = transforms.Compose([transforms.Resize((640, 640)),
                                transforms.ToTensor()])
image = transform(image)


# perform inference
results = model(image)

# inference with larger input size
results = model(image, size=640)

# inference with test time augmentation
results = model(image, augment=True)

# parse results
predictions = results.pred[0]
boxes = predictions[:, :4] # x1, y1, x2, y2
scores = predictions[:, 4]
categories = predictions[:, 5]

# show detection bounding boxes on image
results.show()

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Load the model from Hugging Face
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set the model to evaluation mode
model.eval()

# Load and preprocess the image
image = Image.open('./test_image.jpg')
transform = transforms.Compose([transforms.Resize((640, 640)),
                                transforms.ToTensor()])
image = transform(image)

# Perform object detection
results = model(image)

# Print the predicted labels and bounding boxes
for label, bbox in zip(results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()):
    print(f'Label: {label}, Bounding Box: {bbox}')

# Show the annotated image
results.show()


In [ ]:
# read the file with data using openpyxl
import pandas as pd

# we read the data from the excel file, 
# which is the defect data from the ant 1.3 system
dfDataCamel12 = pd.read_excel('./chapter_6_dataset_numerical.xlsx', 
                            sheet_name='camel_1_2',
                            index_col=0)

In [ ]:
dfDataCamel12

In [ ]:
# prepare the dataset
import sklearn.model_selection

X = dfDataCamel12.drop(['Defect'], axis=1)
y = dfDataCamel12.Defect

X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=42, train_size=0.9)

In [ ]:
y_train

In [ ]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier()
cls.fit(X_train, y_train)
predictions = cls.predict(X_test)

In [ ]:
# now we can get the best models with their weights
cls.leaderboard()

In [ ]:
# and we can even take a look at the best models' scores
ensemble_dict = cls.show_models()
print(ensemble_dict)